In [2]:
# ========================
# 0. IMPORTS
# ========================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Dense, LSTM, RepeatVector, TimeDistributed,
                                    MultiHeadAttention, LayerNormalization, Dropout, GlobalAveragePooling1D,
                                    Conv1D)
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.python.client import device_lib
import os
import gc
import tensorflow.keras.backend as K
# ========================
# 1. CONFIGURATION
# ========================
INPUT_STEPS = 10
FORECAST_STEPS = 10
TEST_RATIO = 0  # For validation split during training
TEST_SIZE_POURCENTAGE = 1
# Tuning parameters
EPOCHS_LIST = [20]
BATCH_SIZES = [128]
WINDOW_SIZE_SIMULATION = 10  # 6h window
# Transformer parameters for anomaly detection
EMBED_DIM = 128
NUM_HEADS = 4
FF_DIM = 256
DROPOUT_RATE = 0.1
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ========================
# 2. DEVICE SETUP
# ========================
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print("✅ GPU is available and will be used.")
    except RuntimeError as e:
        print(e)
else:
    print("⚠️ No GPU detected, running on CPU.")

# ========================
# 3. LOAD AND PREPROCESS DATA
# ========================
# Load detection dataset
file_path_detection = '../../data/cleaned_labeled_dataset.csv'
df_detection = pd.read_csv(file_path_detection, delimiter=',')
df_detection['DateTime'] = pd.to_datetime(df_detection['DateTime'], errors='coerce')
df_detection.set_index('DateTime', inplace=True)

# Load forecast dataset
file_path_forecast = '../../data/preprocessed_data.csv'
df_forecast = pd.read_csv(file_path_forecast, delimiter=',')
df_forecast['DateTime'] = pd.to_datetime(df_forecast['DateTime'], errors='coerce')
df_forecast.set_index('DateTime', inplace=True)

# Extract labels column if it exists from detection dataset
if 'labels' in df_detection.columns:
    print("✅ Found 'labels' column in detection dataset. Extracting for later evaluation.")
    labels_detection = df_detection['labels'].copy()
    df_detection_train = df_detection.drop(columns=['labels'])
else:
    print("⚠️ No 'labels' column found in detection dataset. Will assume all samples are normal.")
    labels_detection = pd.Series(np.zeros(len(df_detection)))
    df_detection_train = df_detection.copy()

# Extract labels column if it exists from forecast dataset
if 'labels' in df_forecast.columns:
    print("✅ Found 'labels' column in forecast dataset. Extracting for later evaluation.")
    labels_forecast = df_forecast['labels'].copy()
    df_forecast_train = df_forecast.drop(columns=['labels'])
else:
    print("⚠️ No 'labels' column found in forecast dataset. Will assume all samples are normal.")
    labels_forecast = pd.Series(np.zeros(len(df_forecast)))
    df_forecast_train = df_forecast.copy()

# ========================
# 4. LOAD TEST SET
# ========================
# Load the test set from CSV
test_set_path = '../../data/test_set.csv'
df_test = pd.read_csv(test_set_path, delimiter=',')
df_test['DateTime'] = pd.to_datetime(df_test['DateTime'], errors='coerce')
df_test.set_index('DateTime', inplace=True)
print(f"✅ Loaded test set with {len(df_test)} samples from {test_set_path}")

# Extract labels from test set if they exist
if 'labels' in df_test.columns:
    print("✅ Found 'labels' column in test set.")
    all_test_labels = df_test['labels'].copy()
    df_test_features = df_test.drop(columns=['labels'])
else:
    print("⚠️ No 'labels' column found in test set. Will assume all samples are normal.")
    all_test_labels = pd.Series(np.zeros(len(df_test)))
    df_test_features = df_test.copy()

# ========================
# 5. NORMALIZE DATA
# ========================
# Normalize detection dataset
scaler_detection = MinMaxScaler()
scaled_data_detection = scaler_detection.fit_transform(df_detection_train.values)
df_detection_scaled = pd.DataFrame(scaled_data_detection, index=df_detection_train.index, 
                                  columns=df_detection_train.columns).astype(np.float32)
print(f"✅ Scaled detection dataset shape: {df_detection_scaled.shape}")

# Normalize forecast dataset
scaler_forecast = MinMaxScaler()
scaled_data_forecast = scaler_forecast.fit_transform(df_forecast_train.values)
df_forecast_scaled = pd.DataFrame(scaled_data_forecast, index=df_forecast_train.index, 
                                 columns=df_forecast_train.columns).astype(np.float32)
print(f"✅ Scaled forecast dataset shape: {df_forecast_scaled.shape}")

# Normalize test dataset - using the same scaler as detection dataset for consistency
scaled_data_test = scaler_detection.transform(df_test_features.values)
test_data_scaled = pd.DataFrame(scaled_data_test, index=df_test_features.index, 
                             columns=df_test_features.columns).astype(np.float32)
print(f"✅ Scaled test dataset shape: {test_data_scaled.shape}")

# ========================
# 6. SEQUENTIAL TRAIN/TEST SPLIT FOR MODEL EVALUATION
# ========================
# Split for detection model
split_idx_detection = int((1 - TEST_RATIO) * len(df_detection_scaled))
train_data_detection = df_detection_scaled.iloc[:split_idx_detection]

# Split for forecast model
split_idx_forecast = int((1 - TEST_RATIO) * len(df_forecast_scaled))
train_data = df_forecast_scaled.iloc[:split_idx_forecast]

✅ GPU is available and will be used.
✅ Found 'labels' column in detection dataset. Extracting for later evaluation.
✅ Found 'labels' column in forecast dataset. Extracting for later evaluation.
✅ Loaded test set with 242614 samples from ../../data/test_set.csv
✅ Found 'labels' column in test set.
✅ Scaled detection dataset shape: (373047, 26)
✅ Scaled forecast dataset shape: (373047, 26)
✅ Scaled test dataset shape: (242614, 26)


NameError: name 'split_idx' is not defined

In [ ]:
# ========================
# 5. CREATE SEQUENCES
# ========================
def create_sequences(data, input_steps, forecast_steps):
    X, y = [], []
    for i in range(len(data) - input_steps - forecast_steps):
        X.append(data[i:i+input_steps])
        y.append(data[i+input_steps:i+input_steps+forecast_steps])
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)


# Create sequence labels (a sequence is anomalous if any point in it is anomalous)
def create_sequence_labels(labels, input_steps, forecast_steps):
    """
    Create sequence-level labels from point-level labels.
    A sequence is considered anomalous (1) if any point in its forecast window is anomalous.
    
    Args:
        labels: Array of point-level binary labels (0=normal, >0=anomaly)
        input_steps: Number of input steps (not used for labeling)
        forecast_steps: Number of forecast steps
        
    Returns:
        Array of sequence-level binary labels
    """
    seq_labels = []
    for i in range(len(labels) - input_steps - forecast_steps):
        # If any point in the forecast window is anomalous, mark the sequence as anomalous
        forecast_window_labels = labels[i+input_steps:i+input_steps+forecast_steps]
        # Convert any non-zero value to anomaly (1)
        is_anomalous = np.any(np.array(forecast_window_labels) > 0)
        seq_labels.append(1 if is_anomalous else 0)
    return np.array(seq_labels, dtype=np.int32)

In [29]:
TEST_SIZE_POURCENTAGE = 0.7
# Split for test set
split_idx_test = int(TEST_SIZE_POURCENTAGE * len(test_data_scaled))
test_data = test_data_scaled.iloc[:split_idx_test]

test_labels = all_test_labels.iloc[:split_idx_test]
print(f"✅ Detection model - Training samples: {len(train_data_detection)}, Testing samples: {len(test_data)}")
print(f"✅ Forecast model - Training samples: {len(train_data)}, Testing samples: {len(test_data)}")

✅ Detection model - Training samples: 373047, Testing samples: 242614
✅ Forecast model - Training samples: 373047, Testing samples: 242614


In [23]:
# ========================
# 8. BUILD AND TRAIN TRANSFORMER AUTOENCODER FOR ANOMALY DETECTION
# ========================
def create_ae_sequences(data, seq_len):
    return np.array([data[i:i+seq_len] for i in range(len(data) - seq_len)], dtype=np.float32)

X_ae_train = create_ae_sequences(train_data_detection.values, FORECAST_STEPS)

def transformer_encoder(inputs, embed_dim, num_heads, ff_dim, dropout_rate=0.1):
    # Multi-head self-attention
    attention_output = MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_dim)(inputs, inputs)
    attention_output = Dropout(dropout_rate)(attention_output)
    attention_output = LayerNormalization(epsilon=1e-6)(inputs + attention_output)
    
    # Feed-forward network
    ffn_output = Dense(ff_dim, activation="relu")(attention_output)
    ffn_output = Dense(embed_dim)(ffn_output)
    ffn_output = Dropout(dropout_rate)(ffn_output)
    
    # Add & Norm
    return LayerNormalization(epsilon=1e-6)(attention_output + ffn_output)

def build_transformer_autoencoder(input_steps, input_dim, embed_dim=128, num_heads=4, ff_dim=256, dropout_rate=0.1):
    inputs = Input(shape=(input_steps, input_dim))
    
    # Initial projection to embed_dim
    x = Conv1D(filters=embed_dim, kernel_size=1, activation='relu')(inputs)
    
    # Encoder: Transformer blocks
    x = transformer_encoder(x, embed_dim, num_heads, ff_dim, dropout_rate)
    x = transformer_encoder(x, embed_dim, num_heads, ff_dim, dropout_rate)
    
    # Bottleneck
    encoded = GlobalAveragePooling1D()(x)
    
    # Decoder: Expand to sequence
    x = RepeatVector(input_steps)(encoded)
    
    # Decoder: Transformer blocks
    x = transformer_encoder(x, embed_dim, num_heads, ff_dim, dropout_rate)
    x = transformer_encoder(x, embed_dim, num_heads, ff_dim, dropout_rate)
    
    # Output projection back to original dimensions
    outputs = TimeDistributed(Dense(input_dim))(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# Build and train the transformer autoencoder for anomaly detection
transformer_ae = build_transformer_autoencoder(
    FORECAST_STEPS, 
    X_ae_train.shape[2],
    embed_dim=EMBED_DIM,
    num_heads=NUM_HEADS,
    ff_dim=FF_DIM,
    dropout_rate=DROPOUT_RATE
)

# Train Transformer AE
es = EarlyStopping(patience=5, restore_best_weights=True)
transformer_ae.fit(
    X_ae_train, 
    X_ae_train, 
    validation_split=0.1, 
    epochs=20, 
    batch_size=128, 
    callbacks=[es], 
    verbose=1
)

#transformer_ae.save("best_transformer_autoencoder.h5")
print("\n✅ Transformer Autoencoder trained and saved.")

Epoch 1/7
2623/2623 [==============================] - 215s 77ms/step - loss: 0.0076 - val_loss: 0.0022
Epoch 2/7
2623/2623 [==============================] - 200s 76ms/step - loss: 7.1328e-04 - val_loss: 0.0016
Epoch 3/7
2623/2623 [==============================] - 194s 74ms/step - loss: 5.9179e-04 - val_loss: 0.0014
Epoch 4/7
2623/2623 [==============================] - 194s 74ms/step - loss: 5.4592e-04 - val_loss: 0.0014
Epoch 5/7
2623/2623 [==============================] - 188s 72ms/step - loss: 0.0022 - val_loss: 0.0107
Epoch 6/7
2623/2623 [==============================] - 191s 73ms/step - loss: 6.4040e-04 - val_loss: 0.0029
Epoch 7/7
2623/2623 [==============================] - 192s 73ms/step - loss: 0.0015 - val_loss: 0.0352

✅ Transformer Autoencoder trained and saved.


In [30]:
K.clear_session()
gc.collect()

83868

In [1]:
# Use the last 3000 points of the test data for simulation
test_data_tail = test_data.tail(10000).reset_index(drop=True)
test_labels_tail = test_labels.tail(10000).reset_index(drop=True)

# ========================
# 9. REAL-TIME SIMULATION WITH MANUAL THRESHOLD CONTROL
# ========================
simulation_X, simulation_y = create_sequences(test_data_tail.values, INPUT_STEPS, FORECAST_STEPS)
# Create corresponding labels for evaluation
simulation_labels = create_sequence_labels(test_labels_tail.values, INPUT_STEPS, FORECAST_STEPS)

#forecast_list = []
#reconstruction_list = []
reconstruction_errors = []
#true_windows = []

# Process all samples without overlapping
step_size = WINDOW_SIZE_SIMULATION  # Use the window size as step size to avoid overlap
for i in range(0, len(simulation_X), step_size):
    # Get the current window batch (up to step_size samples)
   # window_X = simulation_X[i:i+step_size]
    window_y_true = simulation_y[i:i+step_size]
    y_true = window_y_true
    if len(window_X) == 0:
        continue
    
    # Use LSTM Seq2Seq for forecasting
    #y_pred_future = best_model.predict(window_X, batch_size=128, verbose=1)
    #X_forecast = y_pred_future  # No need to expand dims as we're processing batches
    
    # Use Transformer Autoencoder for anomaly detection
    y_reconstructed = transformer_ae.predict(window_y_true, batch_size=128, verbose=1)
    
    # Calculate reconstruction error for each sample in the batch
    batch_reconstruction_errors = np.mean((y_true - y_reconstructed)**2, axis=(1, 2))
    
    # Store predictions and errors
    #forecast_list.append(y_pred_future)
    #reconstruction_list.append(y_reconstructed)
    reconstruction_errors.append(batch_reconstruction_errors)
    #true_windows.append(window_y_true)

print("\n✅ Real-time simulation complete.")

NameError: name 'test_data' is not defined

In [ ]:
MANUAL_THRESHOLD=None
MANUAL_PERCENTILE=90
# Flatten the reconstruction errors list for threshold calculation
all_reconstruction_errors = np.concatenate(reconstruction_errors)

# Make sure we're only considering windows where we have both predictions and labels
min_length = min(len(simulation_labels), len(all_reconstruction_errors))
true_labels_subset = simulation_labels[:min_length]
errors_subset = all_reconstruction_errors[:min_length]

# Determine threshold to use
if MANUAL_THRESHOLD is not None:
    # Use manually specified threshold
    threshold = MANUAL_THRESHOLD
    print(f"\n✅ Using manually specified threshold: {threshold:.5f}")
else:
    # Use percentile-based threshold
    threshold = np.percentile(all_reconstruction_errors, MANUAL_PERCENTILE)
    print(f"\n✅ Using {MANUAL_PERCENTILE}th percentile threshold: {threshold:.5f}")

# Apply threshold to get anomaly flags
anomaly_flags_list = [errors > threshold for errors in reconstruction_errors]
all_detected = np.concatenate([flags for flags in anomaly_flags_list])

# Make sure we're only considering windows where we have both predictions and labels
min_length = min(len(true_labels_subset), len(all_detected))
true_labels_subset = true_labels_subset[:min_length]
all_detected_subset = all_detected[:min_length]

# Print summary of anomalies
print(f"\nAnomaly detection summary:")
print(f"Number of true anomalies: {np.sum(true_labels_subset)}")
print(f"Numberof detected anomalies: {np.sum(all_detected_subset)}")

# Compute metrics
precision = precision_score(true_labels_subset, all_detected_subset, zero_division=0)
recall = recall_score(true_labels_subset, all_detected_subset, zero_division=0)
f1 = f1_score(true_labels_subset, all_detected_subset, zero_division=0)

print(f"\n📈 Anomaly Detection Evaluation:")
print(f"Precision: {precision:.5f}")
print(f"Recall:    {recall:.5f}")
print(f"F1 Score:  {f1:.5f}")

# ========================
# 10. PLOT RESULTS
# ========================
# Plot Reconstruction Errors with True Labels
plt.figure(figsize=(14,5))

# Ensure we're only plotting up to the minimum length we have data for
min_plot_len = min(len(all_reconstruction_errors), len(true_labels_subset))
plot_errors = all_reconstruction_errors[:min_plot_len]
plot_labels = true_labels_subset[:min_plot_len]
plot_detected = all_detected_subset[:min_plot_len]

plt.plot(plot_errors, label='Reconstruction Error')
plt.axhline(threshold, color='red', linestyle='--', label=f'Threshold: {threshold:.5f}')

# Detected anomalies
detected_indices = np.where(plot_detected == 1)[0]
plt.scatter(detected_indices,
            plot_errors[detected_indices],
            color='red', label='Detected Anomalies', s=10)

# True anomalies
true_anomaly_indices = np.where(plot_labels == 1)[0]
if len(true_anomaly_indices) > 0:
    plt.scatter(true_anomaly_indices,
                np.ones_like(true_anomaly_indices) * np.max(plot_errors)*0.9,
                color='green', marker='*', label='True Anomalies', s=20)
else:
    print("No true anomalies found in the subset of data being visualized")

plt.title("Reconstruction Errors vs True Anomalies")
plt.xlabel("Sample Index")
plt.ylabel("Error")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("reconstruction_errors_with_threshold.png")
plt.show()

print("\n✅ Analysis complete.")
print("\nTo change the threshold:")
print("1. Set MANUAL_THRESHOLD to a specific value to use that exact threshold")
print("2. Set MANUAL_PERCENTILE to use a percentile-based threshold (current: {}th)".format(MANUAL_PERCENTILE))
print("3. Re-run the script to see results with the new threshold")